# AdvanTex - manually update to latest release

- Download latest release from [MyAdvanTex](https://www.myadvantex.com/)
- Launch and extract using installer; find file "update.zip" in "Data\\Temp" directory and extract
- Save necessary data in backup tables

In [ ]:
-- backup table "Mandant"
USE Master
GO

SELECT Mandant.ID, Mandant.Bez, Mandant.ConnectPath, Mandant.TestMandant, Mandant.Verfuegbar, Mandant.Anlage_, Mandant.FertigEingespielt, Mandant.DataPath, Mandant.SQLConvertDateFormat, Mandant.SQLConvertTimeFormat 
INTO _Mandant 
FROM dbSystem.dbo.Mandant;

-- Backup Users
USE dbSystem
GO

SELECT db.name
INTO _User
FROM sys.database_principals db, sys.server_principals s 
WHERE (db.Type = 'S' -- SQL_USER
    OR db.Type = 'G' -- Windows Group
    OR db.Type = 'U' -- Windows User
  ) 
  AND db.sid = s.sid;

GO

USE Master
GO

SELECT * INTO _User FROM dbSystem.dbo._User;
GO

- Restore DB "dbsystem" from downloaded release
- Restore from backup tables

In [ ]:
-- Restore date in table "Mandant"
USE dbSystem
GO

TRUNCATE TABLE Mandant;
GO

INSERT INTO Mandant (ID, Bez, ConnectPath, TestMandant, Verfuegbar, Anlage_, FertigEingespielt, DataPath, SQLConvertDateFormat, SQLConvertTimeFormat)
SELECT _Mandant.ID, _Mandant.Bez, _Mandant.ConnectPath, _Mandant.TestMandant, _Mandant.Verfuegbar, _Mandant.Anlage_, _Mandant.FertigEingespielt, _Mandant.DataPath, _Mandant.SQLConvertDateFormat, _Mandant.SQLConvertTimeFormat 
FROM Master.dbo._Mandant;

GO

-- Restore DB-Users
-- Execute SQL, copy result set and execute those statements
USE dbSystem
GO

SELECT 'CREATE USER [' + Name + '] FOR LOGIN [' + Name + '] WITH DEFAULT_SCHEMA = dbo; ' + char(13) + char(10) +
  'ALTER ROLE db_accessadmin ADD MEMBER [' + Name + ']; ' + char(13) + char(10) +
  'ALTER ROLE db_backupoperator ADD MEMBER [' + Name + ']; ' + char(13) + char(10) +
  'ALTER ROLE db_datareader ADD MEMBER [' + Name + ']; ' + char(13) + char(10) +
  'ALTER ROLE db_datawriter ADD MEMBER [' + Name + ']; ' + char(13) + char(10) +
  'ALTER ROLE db_ddladmin ADD MEMBER [' + Name + ']; ' + char(13) + char(10) +
  'ALTER ROLE db_securityadmin ADD MEMBER [' + Name + ']; ' + char(13) + char(10) +
  'ALTER ROLE db_owner ADD MEMBER [' + Name + ']; ' + char(13) + char(10) +
  'GRANT EXECUTE TO [' + Name + ']; ' + char(13) + char(10) +
  'GRANT SUBSCRIBE QUERY NOTIFICATIONS TO [' + Name + '] ' 
FROM Master.dbo._User 
WHERE Upper(Name) <> 'ADSSYS'
  AND Upper(Name) <> 'DBO';

GO

-- Re-activate Service Broker
ALTER DATABASE [dbSystem] SET NEW_BROKER WITH ROLLBACK IMMEDIATE;
GO

-- Change db owner to "adssys"
IF (SELECT COUNT(*) anz FROM sys.server_principals WHERE lower(name) = 'adssys') > 0 
  ALTER AUTHORIZATION ON DATABASE::[dbSystem] TO ADSSYS;

GO

-- Cleanup master database
USE Master
GO

DROP TABLE _Mandant;
DROP TABLE _User;

GO

- Copy remaining files (.exe, .dll, report-files, ...) to folders overwriting existing files
- Run AdvanTex